<a href="https://colab.research.google.com/github/NamanMohindra/EnronClassifier-/blob/master/(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import re
import os
import numpy as np
import random
from numpy import dot
from numpy.linalg import norm

In [0]:
ENRONFOLDER = 'drive/My Drive/Enron Spam/'
ENRONNOSPAMDATASET = 'drive/My Drive/Enron Spam/ham'
FEATURES_LOC = 'drive/My Drive/Enron Spam/featureVectors.npy'

feature_vectors = np.load(FEATURES_LOC)

#initalising the values to 0 of the centroids of formal and informal emails 
formal_flag = 0
informal_flag = 0
formal_centroid = np.zeros(1000)
informal_centroid = np.zeros(1000)
learning_rate = 0.1

annotation_indices = []
annotations = []

In [0]:
def getLabel(mailIndex):
  file_location = os.listdir(ENRONNOSPAMDATASET)[mailIndex]
  FILENAME = os.path.join(ENRONNOSPAMDATASET, file_location)
  with open(FILENAME, 'r',encoding='utf-8',errors='ignore') as myfile:
    data = myfile.read()
    print("Message Contents: ")
    print(data)
    print("Is the displayed message informal[1] or formal[0] ?")
    val = int(input("Enter your value: "))
    return val


def centroid_update(anno_index, label_value):
  global formal_flag
  global informal_flag
  global formal_centroid
  global informal_centroid
  global learning_rate
  fvector = feature_vectors[anno_index]
  f_flag = formal_flag
  if_flag = informal_flag
  if((f_flag == 0) and (label_value==0)):
    formal_flag = 1
    formal_centroid = fvector
  elif((if_flag == 0) and (label_value==1)):
    informal_flag = 1
    informal_centroid = fvector
    #print(fvector)
    #print(informal_centroid)
  elif((f_flag ==1) and (label_value==0)):
    diff = np.subtract(fvector, formal_centroid)
    mult = np.multiply(diff, learning_rate)
    formal_centroid = np.add(mult,formal_centroid)
  elif((if_flag ==1) and (label_value==1)):
    diff = np.subtract(fvector, informal_centroid)
    mult = np.multiply(diff, learning_rate)
    informal_centroid = np.add(mult,informal_centroid)
  else:
    print("Input Error")

def cosineSim(a, b):
  cos_sim = dot(a, b)/(norm(a)*norm(b))
  return cos_sim

def generateIndex(max_val):
  formals = 0
  informals = 0
  for j in annotations:
    if j == 1:
      informals += 1
    if j == 0:
      formals += 1
  if (formals == informals):
    genIndex = random.randrange(max_val)
  else:
    vector_indices = []
    for i in range(10):
      vector_indices.append(random.randrange(max_val))
    index_features = [feature_vectors[x] for x in vector_indices]
    if (formals > informals):
      cos_sims = [cosineSim(y,formal_centroid) for y in index_features]
    else:
      cos_sims = [cosineSim(y,informal_centroid) for y in index_features]
    print(cos_sims)
    genInd = cos_sims.index(min(cos_sims))
    genIndex = vector_indices[genInd]
  return genIndex

In [0]:
def annotateLabels(num_labels):
  for i in range(num_labels):
    anno_index = generateIndex(3300)
    annotation_indices.append(anno_index)
    label_value = getLabel(anno_index)
    annotations.append(label_value)
    centroid_update(anno_index, label_value)

In [0]:
annotateLabels(10)


Message Contents: 
Subject: credit watch list - - week of 11 / 19 / 01
attached is a revised credit watch listing for the week of 11 / 19 / 01 . the yuma companies ( inc . ) has been added to this week ' s " call credit " column .
if there are any personnel in your group that were not included in this distribution , please insure that they receive a copy of this report .
to add additional people to this distribution , or if this report has been sent to you in error , please contact veronica espinoza at x 6 - 6002 .
for other questions , please contact jason r . williams at x 5 - 3923 , veronica espinoza at x 6 - 6002 or darren vanek at x 3 - 1436 .
Is the displayed message informal[1] or formal[0] ?
Enter your value: 1
[0.030261376633440123, 0.07412493166611012, 0.07412493166611012, 0.09513029883089882, 0.06362847629757779, 0.0, 0.11826247919781652, 0.1853123291652753, 0.17118419700436519, 0.12267912301642962]
Message Contents: 
Subject: lyondell - - # 1063
the gas looks like it is not

In [0]:
print(annotation_indices)
print(annotations)

[929, 3093, 82, 1497, 2287, 1599, 3284, 2315, 1988, 2611]
[1, 1, 1, 1, 1, 1, 0, 1, 0, 0]
